# LangChain AI Interview Agent
This notebook provides an overview of an AI interview agent designed for generating interview questions and gathering candidate information based on job requirements, company profile, and candidate resume.

## Setting Up
Uncomment to install the package

In [164]:
# pip install -U "langgraph>=0.2,<0.3" "langchain>=0.3,<0.4" "langchain-openai>=0.2,<0.3"

Uncomment if API key is not added yet

In [165]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

## Step 1: Generate Interview Questions

In [192]:
JOB_POST = """
Position Title: Software Engineer (Backend)

Description:
We are looking for a skilled Software Engineer to join our backend development team. The candidate will work on designing, building, and maintaining scalable APIs and microservices to support our growing business. The role involves collaborating closely with cross-functional teams including front-end developers, product managers, and DevOps engineers.

Key Responsibilities:
- Design, develop, and maintain APIs and backend services.
- Collaborate with cross-functional teams to deliver new features.
- Write clean, efficient, and testable code.

Minimum Requirements:
- 3+ years of experience with backend development.
- Proficiency in Python, Java, or Go.
- Experience with REST APIs and microservices architecture.
"""

COMPANY_PROFILE = """
Company Name: InnovateTech Solutions

Mission: To drive innovation by providing advanced tech solutions that empower businesses to scale and grow.

Vision: To be the leading technology solutions provider, helping businesses navigate the digital transformation with cutting-edge software and exceptional customer service.

Core Values:
1. Innovation: Constantly seeking new ways to solve problems and improve technology.
2. Collaboration: Working together as a team to achieve shared goals.
3. Integrity: Doing the right thing for our customers and employees.
4. Customer Success: Putting customer success at the forefront of every decision we make.
5. Excellence: Striving to deliver the best in everything we do.
"""

CANDIDATE_RESUME = """
Name: John Doe
Email: john.doe@example.com

Summary:
Passionate Software Engineer with 4+ years of experience in backend development. Proven track record of building and optimizing high-performance APIs and microservices. Adept in working with cross-functional teams and improving system security.

Professional Experience:

Backend Engineer | TechSolutions Inc.
January 2020 - Present

	•	Developed and maintained scalable REST APIs using Python and Flask.
	•	Worked with cross-functional teams to deliver new features and product updates.
	•	Implemented security measures like token-based authentication and encryption.
	•	Optimized database queries and improved application performance by 30%.
	•	Assisted in the migration of services to AWS, ensuring high availability and scalability.

Software Engineer | CodeFactory Labs
June 2018 - December 2019

	•	Built microservices in Go to support large-scale applications.
	•	Collaborated with front-end teams to design and integrate APIs.
	•	Improved error handling and logging systems to enhance troubleshooting efficiency.
	•	Used Docker and Kubernetes for containerization and deployment of applications.

Education:
Bachelor’s Degree in Computer Science, University of California, Berkeley, 2017
"""

In [193]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional, List

model = ChatOpenAI(model="gpt-4o")

class InterviewQuestions(BaseModel):
    questions: Optional[List[str]] = Field(
        description="List of interview questions to ask the candidate."
    )

interview_questions_prompt_template = ChatPromptTemplate.from_messages([
    ('system', """You are a recruiter. You are an expert interview question generator. Your task is to generate relevant, targeted interview questions based on the following inputs:
    Job Post: Contains the specific requirements, qualifications, and responsibilities for the position.
    <jobPost>
    {job_post}
    </jobPost>
     
    Company Profile: Provides insights into the company’s values, culture, products, and goals.
    <companyProfile>
    {company_profile}
    </companyProfile>
     
    Candidate Resume: Highlights the candidate’s skills, experience, and accomplishments.
    <candidateResume>
    {candidate_resume}
    </candidateResume>
    """),
    ('user', "Provide 3 (three) thoughtful and personalized interview questions that assess the candidate's fit for the role.")
])

interview_questions_llm_structured_output = interview_questions_prompt_template | model.with_structured_output(InterviewQuestions)
interview_questions_response = interview_questions_llm_structured_output.invoke({"job_post": JOB_POST, "company_profile": COMPANY_PROFILE, "candidate_resume": CANDIDATE_RESUME})
interview_questions = interview_questions_response.questions
print(interview_questions)

['Given your experience working with Python and Flask to develop scalable REST APIs, can you describe a challenging situation you faced in this context and how you overcame it?', 'At InnovateTech Solutions, collaboration is one of our core values. Can you share an experience where you worked closely with cross-functional teams, and how it contributed to the success of a project?', 'We aim to drive innovation at InnovateTech Solutions by constantly seeking new ways to solve problems. Can you tell us about a recent project where you implemented an innovative solution, particularly in the context of improving system security or performance?']


## Building AI Interview Agent

In [194]:
form_result = [{"question_id": index, "interview_question": interview_question, "candidate_answer": ""} for index, interview_question in enumerate(interview_questions)]

class InfoGatheringDetail(BaseModel):
    question_id: int = Field(
        description="The ID of the interview question."
    )
    interview_question: str = Field(
        description="The interview question to ask the candidate"
    )

info_gathering_prompt_template = ChatPromptTemplate.from_messages([
    ('system', """<context>
You are an AI Interview Agent having a conversation with the candidate to gather some required information. You should ask for the information in a natural, conversational way.
</context>

<instructions>
- Do not greet the user, say hi, or introduce yourself because it is assumed that the user is already aware that they are interacting with an AI assistant
- Ask for each piece of information one at a time, waiting for the user's response before moving on to the next item
- Don't list out all the required information upfront
- Phrase your requests as questions that flow naturally in the conversation 
- For output, provide both the question_id and interview_question
- If the user provides information you didn't explicitly ask for yet, acknowledge it and don't ask for it again later
- Once you have collected all the required information or if there is no information to collect (the ask_for list is empty), simply say thank you and tell the user that you have everything that you need for now and you will provide an application update and end the conversation.
</instructions>"""),
    ('user', """<ask_for>
{ask_for}
</ask_for>""")
])
info_gathering_llm = info_gathering_prompt_template | model.with_structured_output(InfoGatheringDetail)

def gather_info(form_result):
    ask_for = [f'<questionId>{item["question_id"]}</questionId><interviewQuestion>{item["interview_question"]}</interviewQuestion>' for item in form_result if item["candidate_answer"] == "" or item["candidate_answer"] == "<UNKNOWN>"]
    info_gathering_response = info_gathering_llm.invoke({"ask_for": ask_for})
    return info_gathering_response

In [195]:
def update_list_if_not_empty(target_list, update_item):
    # Check if 'question_id' and 'candidate_answer' exist in update_item
    if update_item.get('question_id') != None and update_item.get('candidate_answer') != None:
        updated = False
        # Loop through target_list to find the matching 'question_id'
        for item in target_list:
            if item.get('question_id') == update_item.get('question_id'):
                # Update 'candidate_answer' if it's not empty in update_item
                if update_item.get('candidate_answer'):
                    item['candidate_answer'] = update_item['candidate_answer']
                updated = True
                break
        # If no matching 'question_id' is found, add update_item to target_list
        if not updated and update_item.get('candidate_answer'):
            target_list.append(update_item)
    
    return target_list

In [196]:
# Start the conversation with any predefined message
print("AI Interview Agent:", "Hello! I'm an AI Interview Agent and I'm here to help you today. To provide the best assistance, there are a few key pieces of information I need to collect from you first, if that's alright. So to start off, could you please answer the following questions?")
gather_info_response = gather_info(form_result)
print("AI Interview Agent:", gather_info_response.interview_question)

AI Interview Agent: Hello! I'm an AI Interview Agent and I'm here to help you today. To provide the best assistance, there are a few key pieces of information I need to collect from you first, if that's alright. So to start off, could you please answer the following questions?
AI Interview Agent: Given your experience working with Python and Flask to develop scalable REST APIs, can you describe a challenging situation you faced in this context and how you overcame it?


In [197]:
user_input = "One of the most memorable projects I worked on involved developing a scalable REST API for an e-commerce platform using Python and Flask. The API was responsible for handling a high volume of product search queries, user authentication, and order processing, among other features."
form_result = update_list_if_not_empty(form_result, {"question_id": gather_info_response.question_id, "candidate_answer": user_input})
print(form_result)

[{'question_id': 0, 'interview_question': 'Given your experience working with Python and Flask to develop scalable REST APIs, can you describe a challenging situation you faced in this context and how you overcame it?', 'candidate_answer': 'One of the most memorable projects I worked on involved developing a scalable REST API for an e-commerce platform using Python and Flask. The API was responsible for handling a high volume of product search queries, user authentication, and order processing, among other features.'}, {'question_id': 1, 'interview_question': 'At InnovateTech Solutions, collaboration is one of our core values. Can you share an experience where you worked closely with cross-functional teams, and how it contributed to the success of a project?', 'candidate_answer': ''}, {'question_id': 2, 'interview_question': 'We aim to drive innovation at InnovateTech Solutions by constantly seeking new ways to solve problems. Can you tell us about a recent project where you implemente

In [198]:
gather_info_response = gather_info(form_result)
print("AI Interview Agent:", gather_info_response.interview_question)

AI Interview Agent: At InnovateTech Solutions, collaboration is one of our core values. Can you share an experience where you worked closely with cross-functional teams, and how it contributed to the success of a project?


In [199]:
user_input = "I was part of a project where we developed a customer feedback and analytics platform for a SaaS product. This project required close collaboration across several cross-functional teams—front-end developers, product managers, designers, DevOps engineers, and QA."
form_result = update_list_if_not_empty(form_result, {"question_id": gather_info_response.question_id, "candidate_answer": user_input})
print(form_result)

[{'question_id': 0, 'interview_question': 'Given your experience working with Python and Flask to develop scalable REST APIs, can you describe a challenging situation you faced in this context and how you overcame it?', 'candidate_answer': 'One of the most memorable projects I worked on involved developing a scalable REST API for an e-commerce platform using Python and Flask. The API was responsible for handling a high volume of product search queries, user authentication, and order processing, among other features.'}, {'question_id': 1, 'interview_question': 'At InnovateTech Solutions, collaboration is one of our core values. Can you share an experience where you worked closely with cross-functional teams, and how it contributed to the success of a project?', 'candidate_answer': 'I was part of a project where we developed a customer feedback and analytics platform for a SaaS product. This project required close collaboration across several cross-functional teams—front-end developers, 

In [200]:
gather_info_response = gather_info(form_result)
print("AI Interview Agent:", gather_info_response.interview_question)

AI Interview Agent: We aim to drive innovation at InnovateTech Solutions by constantly seeking new ways to solve problems. Can you tell us about a recent project where you implemented an innovative solution, particularly in the context of improving system security or performance?


In [201]:
user_input = "I worked on a project where we aimed to improve the performance and security of a high-traffic e-commerce platform. The platform handled thousands of transactions daily, which exposed it to potential security vulnerabilities and performance bottlenecks, especially during peak shopping periods like Black Friday."
form_result = update_list_if_not_empty(form_result, {"question_id": gather_info_response.question_id, "candidate_answer": user_input})
print(form_result)

[{'question_id': 0, 'interview_question': 'Given your experience working with Python and Flask to develop scalable REST APIs, can you describe a challenging situation you faced in this context and how you overcame it?', 'candidate_answer': 'One of the most memorable projects I worked on involved developing a scalable REST API for an e-commerce platform using Python and Flask. The API was responsible for handling a high volume of product search queries, user authentication, and order processing, among other features.'}, {'question_id': 1, 'interview_question': 'At InnovateTech Solutions, collaboration is one of our core values. Can you share an experience where you worked closely with cross-functional teams, and how it contributed to the success of a project?', 'candidate_answer': 'I was part of a project where we developed a customer feedback and analytics platform for a SaaS product. This project required close collaboration across several cross-functional teams—front-end developers, 

In [203]:
gather_info_response = gather_info(form_result)
print("AI Interview Agent:", gather_info_response.interview_question)

AI Interview Agent: Thank you! I have everything I need for now. I'll provide an application update soon.
